In [3]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader, Dataset

path_OSSE_train = "../data/OSSE_U_V_SLA_SST_train.nc"
path_OSSE_test = "../data/OSSE_U_V_SLA_SST_test.nc"
path_eddies_train = "../data/eddies_train.nc"

eddies_train = xr.open_dataset(path_eddies_train)
OSSE_train = xr.open_dataset(path_OSSE_train)
OSSE_test = xr.open_dataset(path_OSSE_test)

varnames = ["vozocrtxT", "vomecrtyT", "sossheig", "votemper"]
label_var = ["eddies"]


# Data cleaning

Replace NaN values corresponding to lands by 999.

In [ ]:
variables = list(OSSE_train.data_vars.keys())
print("There are {} variables : {}".format(len(variables), variables))
nan_mask = (
    OSSE_train[variables[0]].isnull() &
    OSSE_train[variables[1]].isnull() &
    OSSE_train[variables[2]].isnull() &
    OSSE_train[variables[3]].isnull()
)
OSSE_train_cleaned = OSSE_train.where(~nan_mask, 999)

## MODEL

In [4]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, p = 0.5):
        super(EncoderBlock, self).__init__()

        # Convolution layer n°1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)

        # Batch normalization n°1
        self.bn1 = nn.BatchNorm2d(out_channels)

        # Convolution layer n°2
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

        # Batch normalization n°2
        self.bn2 = nn.BatchNorm2d(out_channels)

        # ReLU activation
        self.relu = nn.ReLU(inplace=True)

        # AlphaDropout layer
        self.alpha_dropout = nn.AlphaDropout(p=p)

        # Max pooling layer for downsampling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        
        # Apply the first convolution + BatchNorm + ReLU activation
        x = self.relu(self.bn1(self.conv1(x)))
        # Apply the second convolution + BatchNorm + ReLU activation
        x = self.relu(self.bn2(self.conv2(x)))
        # Save output for skip connection
        skip_connection = x
        # Apply AlphaDropout before pooling
        x = self.alpha_dropout(x)
        # Apply max pooling to reduce spatial dimensions
        next_layer = self.pool(x)
        return next_layer, skip_connection

In [5]:
class BottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels, p=0.5):
        super(BottleneckBlock, self).__init__()

        # Convolution layer n°1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        
        # Batch normalization n°1
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # Convolution layer n°2
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        
        # Batch normalization n°2
        self.bn2 = nn.BatchNorm2d(out_channels)

        # AlphaDropout Layer
        self.alpha_dropout = nn.AlphaDropout(p=p)

        # ReLU activation
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        
        # Apply convolutions + BatchNorm + ReLU to the input tensor
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.alpha_dropout(x)
        
        return x

In [15]:
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, p=0.5):
        super(DecoderBlock, self).__init__()

        # Transposed convolution for upsampling
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        # Convolution layers for feature refinement
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.alpha_dropout = nn.AlphaDropout(p=p)
        # ReLU activation
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, skip_connection):
        
        # Upsample the input tensor using transposed convolution
        x = self.upconv(x)
        # Concatenate the upsampled tensor with the corresponding encoder skip connection
        x = torch.cat([x, skip_connection], dim=1)
        # Apply convolutions to refine the features
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.alpha_dropout(x)
        x = self.relu(self.bn2(self.conv2(x)))
        return x

In [16]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, init_features=32, p = 0.2):
        super(UNet, self).__init__()
        features = init_features

        # Encoder Path
        self.encoder1 = EncoderBlock(in_channels, features, p=p)
        self.encoder2 = EncoderBlock(features, features * 2, p=p)
        self.encoder3 = EncoderBlock(features * 2, features * 4, p=p)
        self.encoder4 = EncoderBlock(features * 4, features * 8, p=p)

        # Bottleneck
        self.bottleneck = BottleneckBlock(features * 8, features * 16, p=p)

        # Decoder Path
        self.decoder4 = DecoderBlock(features * 16, features * 8, p=p)
        self.decoder3 = DecoderBlock(features * 8, features * 4, p=p)
        self.decoder2 = DecoderBlock(features * 4, features * 2, p=p)
        self.decoder1 = DecoderBlock(features * 2, features, p=p)
        
        # Output Layer
        self.output_conv = nn.Conv2d(features, out_channels, kernel_size=1)

    def forward(self, x):

        # Encoder Path
        x, skip1 = self.encoder1(x)
        x, skip2 = self.encoder2(x)
        x, skip3 = self.encoder3(x)
        x, skip4 = self.encoder4(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Decoder Path
        x = self.decoder4(x, skip4)
        x = self.decoder3(x, skip3)
        x = self.decoder2(x, skip2)
        x = self.decoder1(x, skip1)

        # Output Layer
        output = self.output_conv(x)

        return output

In [ ]:
def train_model(model, train_loader, val_loader=None,
                num_epochs=5, learning_rate =1e-3, device):
    """
    Training function
    
    Args:
        model: PyTorch model
        train_loader: Training data loader
        val_loader: Validation data loader
        num_epochs: Number of training epochs
        learning_rate
        device: Device to train on
    """
    model = model.to(device)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss(ignore_index = -1)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=5)
    
    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        total_train_batches = len(train_loader)

        for batch_idx, (images, masks) in enumerate(train_loader):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

            # Print every batch update
            processed_samples = (batch_idx + 1) * images.size(0)
            total_samples = len(train_loader.dataset)
            percent_complete = 100.0 * (batch_idx + 1) / total_train_batches
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch + 1,
                    processed_samples,
                    total_samples,
                    percent_complete,
                    loss.item(),
                )
            )

        # Average training loss
        train_loss /= total_train_batches
        
        # Optional Validation
        if val_loader:
            model.eval()
            val_loss = 0.0
            total_val_batches = len(val_loader)

            with torch.no_grad():
                for batch_idx, (images, masks) in enumerate(val_loader):
                    images, masks = images.to(device), masks.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, masks)
                    val_loss += loss.item()

                    val_log_str = "Validation: Batch {}/{} \tLoss: {:.6f}".format(
                        batch_idx + 1, total_val_batches, loss.item()
                    )
                
                print(val_log_str)

            val_loss /= total_val_batches
            scheduler.step(val_loss)
        
    return model

In [21]:
class OceanDataset(Dataset):
    def __init__(self, X, Y):
        """
        X : np.ndarray, shape (T, 4, H, W)
        Y : np.ndarray, shape (T, H, W)
        """
        self.X = X
        self.Y = Y
    def __len__(self):
        return self.X.shape[0]  # T
    def __getitem__(self, idx):
        x_arr = self.X[idx]  # shape (4, H, W)
        y_arr = self.Y[idx]  # shape (H, W)
        # Convert to torch.Tensor
        x_torch = torch.from_numpy(x_arr).float()
        y_torch = torch.from_numpy(y_arr).long()  # pour CrossEntropyLoss
        
        return x_torch, y_torch

In [ ]:
# Hyperparameters
LAT = 357
LONG = 717
IMG_SIZE = (LAT, LONG)
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 1

labels_da = eddies_train["eddies"]
Y = labels_da.values
# print("Data type of Y:", Y.dtype)
# print(Y)
Y[np.isnan(Y)] = -1

phys_arrays = []
for v in varnames:
    da_v = OSSE_train[v]
    phys_arrays.append(da_v.values)
X = np.stack(phys_arrays, axis=1)
print("stacked.shape =", X.shape)

model = UNet(in_channels=len(varnames), out_channels=3)

T = X.shape[0]
train_size = int(0.8 * T)
val_size = T - train_size
train_X, val_X = X[:train_size], X[train_size:]
train_Y, val_Y = Y[:train_size], Y[train_size:]
train_dataset = OceanDataset(train_X, train_Y)
val_dataset = OceanDataset(val_X, val_Y)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Train Model
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Lez go Training")
trained_model = train_model(
    model, 
    train_loader,
    val_loader,
    num_epochs=EPOCHS, 
    learning_rate=LEARNING_RATE,
    device=device
)

stacked.shape = (284, 4, 357, 717)
Lez go Training


: 